# Regresion Logistica: Deteccion de Spam

En este ejercicio se muestran los fundamentos de la regresion logistica planteando uno de los primeros problemas que fueron solucionados con el uso de tecnicas de ***Machine Learning***: Deteccion de SPAM.

***La regresion lineal ayuda a predecir eventos a futuro, mientras que la logistica nos ayuda a predecir una probabilidad.***

### Enunciado del ejercicio
Se propone la construccion de un sistema de aprendizaje automatico capaz de predecir si un correo determinado corresponde a un correo ***SPAM*** o no para esto se utilizara el siguiente DataSet: [DataSet](https://www.kaggle.com/datasets/imdeepmind/preprocessed-trec-2007-public-corpus-dataset)

The corpus trec07p contains 75,419 messages:
- 25,220 Ham
- 50,199 SPAM

These messages contitute all the messages delivered to a particular server between these dates 

- Sun, 8 Apr 2007 13:07:21 -0400
- Fri, 6 Jul 2007 07:04:53 -0400


### 1.- Funciones complementarias

En este caso practico relacionado con la deteccion de e-mails de SPAM, el DataSet del que se dispone esta formado por e-mails, con sus correspondientes cabeceras y campos adicionales. Por lo tanto requieren un preprocesamiento previa a que sean ingeridos por el algoritmo de ***Machine Learning***. 

In [17]:
#  Esta clase facilita el preprocesamiento de correos electronicos que poseen codigo HTML 

from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.fed = []

    def handle_data(self,d):
        self.fed.append(d)
    
    def get_data(self):
        return ''.join(self.fed)



In [18]:
# Esta funcion se encarga de eliminar los tags HTML  que se encuentran en el texto del e-mail 

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [19]:
# Ejemplo de eliminacion de los tags HTML de un texto
t='<tr><td aling="left"><a href="../../issues/51/16.html#article">Phrack World News</a></td></tr>'
strip_tags(t)

'Phrack World News'

Ademas de eliminar los posibles tags HTML que se encuentran en el correo electronico, deben de realizarse otras acciones de prepocesamiento para evitar que los mensjes contengan ruido innecesario. Entre ellas se encuentra la eliminacion de los signos de puntuacion, eliminacion de posibles campos de correo electronico que no son reelevantes o eliminacion de afijos de una palabra manteniendo unicamente la raiz de la misma (***Stemming***). La clase que se muestra a continuacion realiza estas transformaciones.

In [20]:
import email
import string
import nltk

In [21]:
class Parser:

    def __init__(self):
        self.stemmer = nltk.PorterStemmer()
        self.stopwords = set(nltk.corpus.stopwords.words('english'))
        self.punctuation = list(string.punctuation)

    def parse(self, email_path):
        """Parse an email."""
        with open(email_path, errors='ignore') as e:
            msg = email.message_from_file(e)
        return None if not msg else self.get_email_content(msg)
    
    def get_email_content(self, msg):
        """Extract the email content"""
        subject = self.tokenize(msg['Subject']) if msg['Subject'] else[]
        body = self.get_email_body(msg.get_payload(),
                                   msg.get_content_type())
        content_type = msg.get_content_type()
        # Returning the content of the email
        return {"subject": subject,
                "body": body,
                "content_type": content_type}
    
    def get_email_body(self, payload, content_type):
        """Extract the body of the email"""
        body = []
        if type(payload) is str and content_type == 'text/plain':
            return self.tokenize(payload)
        elif type(payload) is str and content_type == 'text/html':
            return self.tokenize(strip_tags(payload))
        elif type(payload) is list:
            for p in payload:
                body += self.get_email_body(p.get_payload(),
                                            p.get_content_type())
        return body
    
    def tokenize(self, text):
        """Transform a text string in tokens. perform two main actions,
        clean the punctuation symbols and do stemming of the text."""
        for c in self.punctuation:
            text = text.replace(c, "")
        text = text.replace("\t", " ")
        text = text.replace("\n", " ")
        tokens = list(filter(None, text.split(" ")))
        # Stemming of the tokens
        return [self.stemmer.stem(w) for w in tokens if w not in self.stopwords]



Lectura de un e-mail en formato Raw

In [22]:
inmail = open('./datasets/trec07p/data/inmail.1').read()
print(inmail)

From RickyAmes@aol.com  Sun Apr  8 13:07:32 2007
Return-Path: <RickyAmes@aol.com>
Received: from 129.97.78.23 ([211.202.101.74])
	by speedy.uwaterloo.ca (8.12.8/8.12.5) with SMTP id l38H7G0I003017;
	Sun, 8 Apr 2007 13:07:21 -0400
Received: from 0.144.152.6 by 211.202.101.74; Sun, 08 Apr 2007 19:04:48 +0100
Message-ID: <WYADCKPDFWWTWTXNFVUE@yahoo.com>
From: "Tomas Jacobs" <RickyAmes@aol.com>
Reply-To: "Tomas Jacobs" <RickyAmes@aol.com>
To: the00@speedy.uwaterloo.ca
Subject: Generic Cialis, branded quality@ 
Date: Sun, 08 Apr 2007 21:00:48 +0300
X-Mailer: Microsoft Outlook Express 6.00.2600.0000
MIME-Version: 1.0
Content-Type: multipart/alternative;
	boundary="--8896484051606557286"
X-Priority: 3
X-MSMail-Priority: Normal
Status: RO
Content-Length: 988
Lines: 24

----8896484051606557286
Content-Type: text/html;
Content-Transfer-Encoding: 7Bit

<html>
<body bgcolor="#ffffff">
<div style="border-color: #00FFFF; border-right-width: 0px; border-bottom-width: 0px; margin-bottom: 0px;" align="

Parsear el e-mail

In [23]:
p = Parser()
p.parse("datasets/trec07p/data/inmail.1")

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/home/maury/nltk_data'
    - '/home/maury/Sources/Simulacion/.venv/nltk_data'
    - '/home/maury/Sources/Simulacion/.venv/share/nltk_data'
    - '/home/maury/Sources/Simulacion/.venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


### Lectura del indice

Estas funciones complementarias se encargan de cargar en memoria la ruta de cada correo electronico y su etiqueta correspondiente {ham, spam}.

In [ ]:
index=open("datasets/trec07p/full/index").readlines()
index

In [ ]:
import os 
DATASET_PATH="datasets/trec07p"

def parse_index(path_to_index, n_elements):
    ret_indexes = []
    index = open(path_to_index).readlines()
    for i in range(n_elements):
        mail = index[i].split(" ../")
        label = mail[0]
        path = mail[1][:-1]
        ret_indexes.append({"label": label, "email_path":os.path.join(DATASET_PATH, path)})
    return ret_indexes


In [ ]:
def parse_email(index):
    p=Parser()
    pmail=p.parse(index["email_path"])
    return pmail,index["label"]


In [ ]:
indexes = parse_index("datasets/trec07p/full/index", 10)
indexes

## 2.- Preprocesamiento de los datos del DataSet

Con las funciones presentadas anteriormente se permite la lectura de los correos electronicos de manera programatica y el preprocesamiento de los mismos para eliminar aquellos componentes que no resultan de utilidad para la deteccion de correos SPAM. Sin embargo cada uno de los correos sigue estando representado por un diccionario de Python con una serie de palabras.

In [ ]:
# Cargar el indice y las etiquetas en memoria
index=parse_index('datasets/trec07p/full/index',1)

In [ ]:
# Leer el primer correo 
import os 

open(index[0]['email_path']).readlines()

In [ ]:
# Parser el primer correo 
mail, label = parse_email(index[0])
print("Label:", label)
print(mail)

El algoritmo de regresion logistica no es capaz de ingerir text como parte del dataset.  Por lo tanto deben aplicarse, una serie de funciones adicionales que transforme el texto de los correos parseados en una representacion numerica.

Aplicacion de CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Preparacion del e-mail en una cadena de texto
prep_email = [" ".join(mail['subject']) + " ".join(mail['body'])]

vectorizer = CountVectorizer()
x = vectorizer.fit(prep_email)
print("e-mail:",prep_email, "\n")
print("Caracteristicas de entrada:", vectorizer.get_feature_names_out())

In [ ]:
x = vectorizer.transform(prep_email)
print("\nValues:\n",x.toarray())

In [ ]:
# Creacion de la matriz
from sklearn.preprocessing import OneHotEncoder
prep_email = [[w] for w in mail['subject'] + mail['body']]
enc = OneHotEncoder(handle_unknown='ignore')
x = enc.fit_transform(prep_email)

print("Features: \n",enc.get_feature_names_out())
print("\n Values: \n",x.toarray())

Funciones auxiliares para el preprocesamiento del DataSet

In [ ]:
def create_prep_dataset(index_path,n_elements):
    X=[]
    Y=[]
    indexes=parse_index(index_path,n_elements)
    for i in range(n_elements):
        print("\n Parsing e-mail: {0}".format(i+1),end='')
        mail, label = parse_email(indexes[i])
        X.append(" ".join(mail['subject']) + ' '.join(mail['body']))
        Y.append(label)
    return X,Y

## 3.-Entrenamiento del Algoritmo

In [ ]:
# Leer unicamente un subconjunto de 100 correos

X_train, Y_train = create_prep_dataset('./datasets/trec07p/full/index',100)
X_train

In [ ]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)

In [ ]:
print(X_train.toarray())
print('\nFeatures: ',len(vectorizer.get_feature_names_out()))

In [ ]:
import pandas as pd 

pd.DataFrame(X_train.toarray(),columns=[vectorizer.get_feature_names_out()])

In [ ]:
Y_train

Entrenamiento del algoritmo de regresion logistica con el DataSet preprocesado

In [ ]:
# Entrenar modelo
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train,Y_train)

### 4.- Prediccion 
Lectura de un DataSet de corres electronicos

In [ ]:
# Leer 150 correos electronicos de nuestro DataSet y nos quedamos con los 50 ultimos, esto 50 correos electronicos no se han utilizado 
# para entrenar el algoritmo
X,Y=create_prep_dataset('./datasets/trec07p/full/index',150)
X_test = X[100:]
Y_test = Y[100:]


Preprocesamiento de los correos electronicos con el vectorizador creado anteriormente

In [ ]:
X_test = vectorizer.transform(X_test)

Prediccion del tipo de correo

In [ ]:
Y_pred = clf.predict(X_test)
Y_pred

In [ ]:
print("Prediccion: \n",Y_pred)
print("\nEtiquetas reales: \n",Y_test)

Evaluacion de Resultados

In [ ]:
from sklearn.metrics import accuracy_score
print('Accuracy: {:3f}'.format(accuracy_score(Y_test,Y_pred)))

### 5.- Aumentando el DataSet

In [ ]:
# Leer 12,000 correos electronicos para entrenar el algoritmo y 2,000 para realizar pruebas
X,Y=create_prep_dataset("./datasets/trec07p/full/index",12000)

In [ ]:
X_train,Y_train = X[:10000],Y[:10000]
X_test,Y_test = X[10000:],Y[10000:]

In [ ]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)

In [ ]:
clf = LogisticRegression()
clf.fit(X_train,Y_train)

In [ ]:
X_test = vectorizer.transform(X_test)
y_pred = clf.predict(X_test)
print('Accuracy: {:.3f}'.format(accuracy_score(Y_test,y_pred)))